# LDA Try on Real Data (incomplete)

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

## Sequence Data

In [13]:
annotate_data_file_path='annotate_peaks_GRCh38_hg38/annotate_peaks_data_GRCh38_hg38.tsv'
raw_data=pd.read_csv(annotate_data_file_path,sep='\t',low_memory=False)

In [14]:
peak_id_cols=[col for col in raw_data.columns if 'PeakID' in col]
raw_data.rename(columns={peak_id_cols[0]:'PeakID'},inplace=True)
print(f'Raw-Data Shape = {raw_data.shape}')
raw_data.head()

Raw-Data Shape = (60100, 322)


,PeakID,Chr,Start,End,Strand,Peak Score,Focus Ratio/Region Size,Annotation,Detailed Annotation,Distance to TSS,...,"CRE(bZIP)/Promoter/Homer Distance From Peak(sequence,strand,conservation)","Isl1(Homeobox)/Neuron-Isl1-ChIP-Seq(GSE31456)/Homer Distance From Peak(sequence,strand,conservation)","LHX9(Homeobox)/Hct116-LHX9.V5-ChIP-Seq(GSE116822)/Homer Distance From Peak(sequence,strand,conservation)","Tcf12(bHLH)/GM12878-Tcf12-ChIP-Seq(GSE32465)/Homer Distance From Peak(sequence,strand,conservation)","EHF(ETS)/LoVo-EHF-ChIP-Seq(GSE49402)/Homer Distance From Peak(sequence,strand,conservation)","MYB(HTH)/ERMYB-Myb-ChIPSeq(GSE22095)/Homer Distance From Peak(sequence,strand,conservation)","Foxa3(Forkhead)/Liver-Foxa3-ChIP-Seq(GSE77670)/Homer Distance From Peak(sequence,strand,conservation)","Ascl1(bHLH)/NeuralTubes-Ascl1-ChIP-Seq(GSE55840)/Homer Distance From Peak(sequence,strand,conservation)","LEF1(HMG)/H1-LEF1-ChIP-Seq(GSE64758)/Homer Distance From Peak(sequence,strand,conservation)","JunB(bZIP)/DendriticCells-Junb-ChIP-Seq(GSE36099)/Homer Distance From Peak(sequence,strand,conservation)"
0,seq_40800,chr4,20984016,20984165,+,0,NaN,NaN,NaN,-81,...,0,0,0,0,0,0,0,1,0,0
1,seq_10314,chr11,76444716,76444865,+,0,NaN,NaN,NaN,-228,...,0,0,0,0,0,0,0,0,0,0
2,seq_18866,chr15,67558636,67558785,+,0,NaN,NaN,NaN,9616,...,0,0,0,0,0,0,0,0,0,0
3,seq_45348,chr5,134512248,134512397,+,0,NaN,NaN,NaN,5770,...,0,0,2,0,0,0,0,0,0,0
4,seq_2616,chr1,92804876,92805025,+,0,NaN,NaN,NaN,-12540,...,0,0,0,0,0,1,0,0,0,0


In [15]:
motif_cols=[]
for col in raw_data.columns:
  if '/Homer' in col:
    motif_cols.append(col)
print(f'Total No. of Motif = {len(motif_cols)}')

Total No. of Motif = 301


## Know Motif Data

In [16]:
motif_file_path='output_GRCh38_hg38/knownResults.txt'
motif_data=pd.read_csv(motif_file_path,sep='\t')

In [17]:
print(f'Motif-Data Shape = {motif_data.shape}')
motif_data.head()

Motif-Data Shape = (440, 9)


,Motif Name,Consensus,P-value,Log P-value,q-value (Benjamini),# of Target Sequences with Motif(of 60108),% of Target Sequences with Motif,# of Background Sequences with Motif(of 59450),% of Background Sequences with Motif
0,CTCF(Zf)/CD4+-CTCF-ChIP-Seq(Barski_et_al.)/Homer,AYAGTGCCMYCTRGTGGCCA,0.0,-24440.0,0.0,12286.0,20.44%,713.4,1.20%
1,BORIS(Zf)/K562-CTCFL-ChIP-Seq(GSE32465)/Homer,CNNBRGCGCCCCCTGSTGGC,0.0,-18580.0,0.0,14267.0,23.74%,1764.5,2.98%
2,"OCT4-SOX2-TCF-NANOG(POU,Homeobox,HMG)/mES-Oct4...",ATTTGCATAACAATG,0.0,-3513.0,0.0,3076.0,5.12%,417.5,0.71%
3,Fosl2(bZIP)/3T3L1-Fosl2-ChIP-Seq(GSE56872)/Homer,NATGASTCABNN,0.0,-2532.0,0.0,4418.0,7.35%,1220.3,2.06%
4,Fra2(bZIP)/Striatum-Fra2-ChIP-Seq(GSE43429)/Homer,GGATGACTCATC,0.0,-2500.0,0.0,5365.0,8.93%,1731.3,2.92%


#### Identify Duplicate Motifs (based on their names)

In [18]:
#duplicate motifs
motif_data[motif_data['Motif Name'].duplicated(keep=False)]

,Motif Name,Consensus,P-value,Log P-value,q-value (Benjamini),# of Target Sequences with Motif(of 60108),% of Target Sequences with Motif,# of Background Sequences with Motif(of 59450),% of Background Sequences with Motif
268,RORgt(NR)/EL4-RORgt.Flag-ChIP-Seq(GSE56019)/Homer,AAYTAGGTCA,0.00001,-11.67,0.0,536.0,0.89%,436.5,0.74%
269,RORgt(NR)/EL4-RORgt.Flag-ChIP-Seq(GSE56019)/Homer,AAYTAGGTCA,0.00001,-11.67,0.0,536.0,0.89%,436.5,0.74%


## Preprocessing

### 1. Rename Motif Columns

In [19]:
data=raw_data.copy()
motif_dict={}
for idx,row in motif_data.iterrows():
    motif_dict[row['Motif Name']+' Distance From Peak(sequence,strand,conservation)']='motif_'+str(idx+1)
    
data.rename(columns=motif_dict,inplace=True)
new_motif_cols=motif_dict.values()#[col for col in data.columns if 'motif_' in col]
data[['PeakID']+new_motif_cols].head()

,PeakID,motif_100,motif_101,motif_102,motif_103,motif_104,motif_105,motif_106,motif_107,motif_108,...,motif_91,motif_92,motif_93,motif_94,motif_95,motif_96,motif_97,motif_98,motif_99,motif_9
0,seq_40800,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,seq_10314,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,seq_18866,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,seq_45348,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
4,seq_2616,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


### 2. Create Documents and BOW Corpus

In [20]:
motif_seq=[]
for idx,row in tqdm(data.iterrows()):
    l=[]
    for motif in new_motif_cols:
        l.extend(row[motif]*[motif])
    motif_seq.append(l)
data['motif_seq']=motif_seq

60100it [01:17, 778.03it/s] 


In [21]:
print('Sequence-0:')
data['motif_seq'][0]

Sequence-0:


['motif_149',
 'motif_162',
 'motif_166',
 'motif_166',
 'motif_174',
 'motif_199',
 'motif_230',
 'motif_248',
 'motif_295',
 'motif_295',
 'motif_60',
 'motif_60',
 'motif_66',
 'motif_98']

In [26]:
from gensim.corpora import Dictionary

docs=data['motif_seq']
dictionary=Dictionary(docs)

print('Motif-Index to Motif-Name Mapping:')
for i,v in dictionary.items():
    print(f'{i} - {v}')
    if i==10:
        break
    

Motif-Index to Motif-Name Mapping:
0 - motif_149
1 - motif_162
2 - motif_166
3 - motif_174
4 - motif_199
5 - motif_230
6 - motif_248
7 - motif_295
8 - motif_60
9 - motif_66
10 - motif_98


In [27]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('BOW (Sequence-0):')
corpus[0]

BOW (Sequence-0):


[(0, 1),
 (1, 1),
 (2, 2),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 2),
 (8, 2),
 (9, 1),
 (10, 1)]

### LDA Training (to be implemented..)

In [28]:
#setup logging for trainging metrics 
import logging
logging.basicConfig(filename='test_output/model_callbacks.log', filemode='w',
                    format="%(asctime)s:%(levelname)s:%(message)s",
                    level=logging.NOTSET)

from gensim.models.callbacks import Callback,PerplexityMetric, ConvergenceMetric, CoherenceMetric
perplexity_logger = PerplexityMetric(corpus=corpus, logger='shell')
convergence_logger = ConvergenceMetric(logger='shell')
# coherence_cv_logger = CoherenceMetric(corpus=corpus, coherence = 'c_v', texts = docs)

In [30]:
%%time
from gensim.models import LdaModel,LdaMulticore

#HYPERPARAMETERS
# .
# .
# .
#passes = epochs
temp = dictionary[0]
id2word = dictionary.id2token
lda = LdaModel(corpus, id2word=id2word, alpha='auto',eval_every = 1,\
               eta='auto',num_topics=3, iterations=5, passes = 10,
              callbacks=[perplexity_logger,convergence_logger])

CPU times: user 11min 24s, sys: 677 ms, total: 11min 24s
Wall time: 11min 26s


In [31]:
lda.print_topics()

[(0,
  '0.067*"motif_60" + 0.054*"motif_184" + 0.046*"motif_37" + 0.043*"motif_23" + 0.042*"motif_218" + 0.041*"motif_42" + 0.033*"motif_13" + 0.032*"motif_40" + 0.029*"motif_166" + 0.028*"motif_19"'),
 (1,
  '0.025*"motif_62" + 0.025*"motif_60" + 0.021*"motif_126" + 0.019*"motif_84" + 0.016*"motif_54" + 0.015*"motif_50" + 0.014*"motif_49" + 0.014*"motif_48" + 0.014*"motif_41" + 0.013*"motif_93"'),
 (2,
  '0.054*"motif_219" + 0.044*"motif_11" + 0.043*"motif_12" + 0.043*"motif_10" + 0.039*"motif_9" + 0.038*"motif_6" + 0.036*"motif_264" + 0.035*"motif_5" + 0.033*"motif_8" + 0.028*"motif_4"')]

#### Prediction Example

In [33]:
for doc in corpus[:10]:
    print(sorted(lda[doc],key=lambda x:-x[1]))

[(0, 0.64587086), (1, 0.3444889)]
[(0, 0.56868815), (1, 0.366404), (2, 0.064907864)]
[(1, 0.90899414), (0, 0.08469088)]
[(1, 0.8032862), (0, 0.13044624), (2, 0.06626752)]
[(1, 0.9818696), (0, 0.01371318)]
[(0, 0.7234042), (1, 0.20433637), (2, 0.072259426)]
[(1, 0.77918005), (2, 0.17365634), (0, 0.0471636)]
[(0, 0.63867813), (1, 0.35611957)]
[(2, 0.5852087), (1, 0.39501527), (0, 0.019776009)]
[(0, 0.72819036), (1, 0.2677562)]


#### Plotting Test

In [35]:
data['Strand'].value_counts()

+    60100
Name: Strand, dtype: int64